In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Khởi tạo danh sách các cột
columns = ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
data = []

# Số lượng trang cần thu thập dữ liệu (mỗi trang chứa khoảng 50 dòng)
num_pages = 20

# Lặp qua các trang để thu thập dữ liệu
for page in range(1, num_pages + 1):
    url = f'https://www.amazon.com/primevideo/browse/?_encoding=UTF8&ref_=nav_ya_signin&page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Lặp qua các phần tử chứa dữ liệu trên trang
    for item in soup.find_all('div', class_='av-pinned-content-row'):
        # Khởi tạo một từ điển để lưu trữ thông tin của mỗi dòng dữ liệu
        row_data = {}
        # Thu thập thông tin của mỗi cột
        row_data['show_id'] = item.get('data-asin')
        row_data['type'] = item.find('span', class_='avu-full-width').text.strip()
        row_data['title'] = item.find('h1').text.strip()
        row_data['director'] = item.find('div', class_='av-dnp-image-container').find_next('span').text.strip()
        row_data['cast'] = item.find('div', class_='av-dnp-image-container').find_next('span').find_next('span').text.strip()
        row_data['country'] = item.find('span', class_='avu-full-width').find_next('span').find_next('span').text.strip()
        row_data['date_added'] = item.find('span', class_='avu-full-width').find_next('span').find_next('span').find_next('span').text.strip()
        row_data['release_year'] = item.find('span', class_='av-dnp-detail').find_next('span').text.strip()
        row_data['rating'] = item.find('span', class_='av-dnp-rating').text.strip()
        row_data['duration'] = item.find('span', class_='av-dnp-runtime').text.strip()
        row_data['listed_in'] = item.find('span', class_='av-dnp-genre').text.strip()
        row_data['description'] = item.find('div', class_='av-dnp-episode-synopsis').text.strip()

        # Thêm dòng dữ liệu vào danh sách
        data.append(row_data)

# Tạo DataFrame từ danh sách dữ liệu
df = pd.DataFrame(data, columns=columns)

# Lưu dữ liệu vào file CSV
df.to_csv('DATA.csv', index=False)

In [24]:
import pandas as pd

# Đường dẫn đến file CSV chứa dữ liệu
file_path = '/content/drive/MyDrive/DATA.csv'

# Đọc dữ liệu từ file CSV vào DataFrame
data = pd.read_csv(file_path)

# Hiển thị thông tin cơ bản về dữ liệu
print("Thông tin cơ bản về dữ liệu:")
print(data.info())

# Hiển thị một số dòng đầu tiên của DataFrame
print("\nDữ liệu trước khi làm sạch:")
print(data.head())

# Xử lý và làm sạch dữ liệu
# Loại bỏ các cột không cần thiết
columns_to_drop = ['director', 'cast', 'country', 'date_added']
data_cleaned = data.drop(columns=columns_to_drop)

# Xử lý cột rating: Điền giá trị thiếu bằng 'Unknown'
data_cleaned['rating'].fillna('Unknown', inplace=True)

# Xử lý cột duration: Thay thế các giá trị không hợp lệ bằng NaN (giá trị thiếu)
data_cleaned['duration'] = pd.to_numeric(data_cleaned['duration'], errors='coerce')

# Loại bỏ các giá trị NaN (giá trị thiếu) trong cột duration
data_cleaned = data_cleaned.dropna(subset=['duration'])

# Hiển thị dữ liệu sau khi làm sạch
print("\nDữ liệu sau khi làm sạch:")
print(data_cleaned.head())

# Lưu trữ dữ liệu đã làm sạch vào file CSV mới
cleaned_file_path = 'amazonprime_cleaned.csv'
data_cleaned.to_csv(cleaned_file_path, index=False)

print("\nDữ liệu đã làm sạch được lưu vào file:", cleaned_file_path)


Thông tin cơ bản về dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB
None

Dữ liệu trước khi làm sạch:
  show_id   type                 title        director  \
0      s1  Movie   The Grand Seduction    Don McKellar   
1      s2  Movie  Take Care Good Night    Girish Joshi   
2      s3  Movi